**Table of contents**<a id='toc0_'></a>    
- [三角関数による順運動学](#toc1_)    
- [「2軸の回転関節を持ったロボットアーム」における三角関数による順運動学](#toc2_)    
  - [理論](#toc2_1_)    
    - [リンク1の根本（=原点）からみた時のリンク1の先端位置](#toc2_1_1_)    
    - [リンク2の根本からみた時のリンク2の先端位置（=エンドエフェクタの位置）](#toc2_1_2_)    
    - [リンク1の根本（=原点）から見た時のエンドエフェクタの位置](#toc2_1_3_)    
  - [実装](#toc2_2_)    
    - [pybulletの起動](#toc2_2_1_)    
    - [pybulletの初期設定](#toc2_2_2_)    
    - [ロボットアームの生成](#toc2_2_3_)    
    - [三角関数による順運動学の関数の定義](#toc2_2_4_)    
    - [三角関数による順運動学の実行](#toc2_2_5_)    

<!-- vscode-jupyter-toc-config
	numbering=false
	anchor=true
	flat=false
	minLevel=1
	maxLevel=6
	/vscode-jupyter-toc-config -->
<!-- THIS CELL WILL BE REPLACED ON TOC UPDATE. DO NOT WRITE YOUR TEXT IN THIS CELL -->

# <a id='toc1_'></a>[三角関数による順運動学](#toc0_)

本notebookでは、2軸ロボットアームを用いて「三角関数による順運動学」をPybulletで実装する手順について解説します。

（pybulletで使用可能な関数がまとめられたマニュアルについては[こちら](https://github.com/bulletphysics/bullet3/blob/master/docs/pybullet_quickstartguide.pdf)を参照してください。）

<br>

「三角関数により順運動学」では
- 「各関節の角度 $\theta$」「リンクの長さ $l$」

から

- 「エンドエフェクタの位置 $(x_{\mathrm{e}}, y_{\mathrm{e}})$」

を求めます。

![](../images/RobotArm/robot_arm_trigonometric_forward_kinematics/overview.png)



# <a id='toc2_'></a>[「2軸の回転関節を持ったロボットアーム」における三角関数による順運動学](#toc0_)

ここからは、「2軸の回転関節を持ったロボットアーム」における三角関数による順運動学について理論と実装を解説します。

## <a id='toc2_1_'></a>[理論](#toc0_)

2軸の回転関節を持ったロボットアームの場合、エンドエフェクタの位置は以下の手順で求められます。

一気に「エンドエフェクタの位置 $(x_{\rm{e}}, y_{\rm{e}})$」まで求めても良いですが，ここでは
1. **リンク1の根本（=原点）からみた時の**リンク1の先端位置 $(x_{\rm{1\to 2}}, y_{\rm{1\to 2}})$
2. **リンク2の根本からみた時の**リンク2の先端位置（=エンドエフェクタの位置） $(x_{\rm{2\to e}}, y_{\rm{2\to e}})$
3. **リンク1の根本（=原点）から見た時の**エンドエフェクタの位置$(x_{\rm{e}}, y_{\rm{e}})$

の順番で求めていくことにします．

![](../images/RobotArm/robot_arm_trigonometric_forward_kinematics/2d_arm_forward_kinematics_overview.png)

<br>

### <a id='toc2_1_1_'></a>[リンク1の根本（=原点）からみた時のリンク1の先端位置](#toc0_)

「**リンク1の根本から見た時の**リンク1の先端位置$(x_{\rm{1\to2}},y_{\rm{1\to2}})$」は，
- リンク1の長さを$L_1$
- リンク1の関節角度を$\theta_1$

とした時

$$
\begin{align}
x_{\rm{1\to 2}} &= L_1\cos(\theta_1)\tag{1}
\end{align}
$$

$$
\begin{align}
y_{\rm{1\to 2}} &= L_1\sin(\theta_1)\tag{2}
\end{align}
$$


となります。

![](../images/RobotArm/robot_arm_trigonometric_forward_kinematics/step1.png)

<br>

### <a id='toc2_1_2_'></a>[リンク2の根本からみた時のリンク2の先端位置（=エンドエフェクタの位置）](#toc0_)

次に，
「**リンク2の根本から見た時の**リンク2の先端位置$(x_{2\to e}, y_{2\to e})$」は，
- リンク2の長さを$L_2$
- 各関節の角度を$\theta_1, \theta_2$

とした時

$$
\begin{align}
x_{2\to e} &= L_2\cos(\theta_1+\theta_2)
\end{align}
\tag{3}
$$

$$
\begin{align}
y_{2\to e} &= L_2\sin(\theta_1+\theta_2) 
\end{align}
\tag{4}
$$

となります．

---

---

ここで， $(x_{\rm{2\to e}}, y_{\rm{2\to e}})$ を求める際は，$\theta_1$も考慮する必要があることに注意してください．

---

---

![](../images/RobotArm/robot_arm_trigonometric_forward_kinematics/step2.png)


<br>

### <a id='toc2_1_3_'></a>[リンク1の根本（=原点）から見た時のエンドエフェクタの位置](#toc0_)

ここまでに求めた位置を足し合わせることで，「**リンク1の根本（=原点）から見た時の**エンドエフェクタの位置$(x_{\rm{e}}, y_{\rm{e}})$」を求めることができます．

$$
\begin{align}
x_{\rm{e}} &= x_{\rm{1\to 2}} + x_{\rm{2\to e}} \tag{5}
\end{align}
$$

$$
\begin{align}
y_{\rm{e}} &= y_{\rm{1\to 2}} + y_{\rm{2\to e}} \tag{6}
\end{align}
$$


---

---

多くのロボットアームにおいて，
- 「$i$番目のリンクの根本」と「$i+1$番目のリンクの先端」の位置

は一致しているため，上記の式のように単純に足し合わせることで**リンク1の根本（=原点）からみた時の**エンドエフェクタの位置を求めることができます．もしも，一致していない場合はそれを考慮する必要があるので注意してください．

![](../images/RobotArm/robot_arm_trigonometric_forward_kinematics/note_link_root.png)

---

---

<br>

## <a id='toc2_2_'></a>[実装](#toc0_)
ここからは、実際にPybulletで「2軸の回転関節を持ったロボットアーム」における三角関数による順運動学を実装していきます。

### <a id='toc2_2_1_'></a>[pybulletの起動](#toc0_)

In [1]:
import pybullet
import pybullet_data
physicsClient = pybullet.connect(pybullet.GUI) 

pybullet build time: Nov 28 2023 23:45:17


startThreads creating 1 threads.
starting thread 0
started thread 0 
argc=2
argv[0] = --unused
argv[1] = --start_demo_name=Physics Server
ExampleBrowserThreadFunc started
X11 functions dynamically loaded using dlopen/dlsym OK!
X11 functions dynamically loaded using dlopen/dlsym OK!
Creating context
Created GL 3.3 context
Direct GLX rendering context obtained
Making context current
GL_VENDOR=Mesa
GL_RENDERER=llvmpipe (LLVM 15.0.7, 256 bits)
GL_VERSION=4.5 (Core Profile) Mesa 23.2.1-1ubuntu3.1~22.04.2
GL_SHADING_LANGUAGE_VERSION=4.50
pthread_getconcurrency()=0
Version = 4.5 (Core Profile) Mesa 23.2.1-1ubuntu3.1~22.04.2
Vendor = Mesa
Renderer = llvmpipe (LLVM 15.0.7, 256 bits)
b3Printf: Selected demo: Physics Server
startThreads creating 1 threads.
starting thread 0
started thread 0 
MotionThreadFunc thread started


### <a id='toc2_2_2_'></a>[pybulletの初期設定](#toc0_)

In [2]:
pybullet.resetSimulation() # シミュレーション空間をリセット
pybullet.setAdditionalSearchPath(pybullet_data.getDataPath()) # pybulletに必要なデータへのパスを追加
pybullet.setGravity(0.0, 0.0, -9.8) # 地球上における重力に設定
timeStep = 1./240.
pybullet.setTimeStep(timeStep) # 1stepあたりに経過する時間の設定

#床の読み込み
planeId = pybullet.loadURDF("plane.urdf")

# GUIモードの際のカメラの位置などを設定
cameraDistance = 2.0
cameraYaw = 0.0 # deg
cameraPitch = -20 # deg
cameraTargetPosition = [0.0, 0.0, 0.0]
pybullet.resetDebugVisualizerCamera(cameraDistance, cameraYaw, cameraPitch, cameraTargetPosition)

ven = Mesa
ven = Mesa


### <a id='toc2_2_3_'></a>[ロボットアームの生成](#toc0_)
今回は、2軸ロボットアーム`simple_2d_arm.urdf`を生成します。  
ロボットは下図のような構成になっています（センサーについては、`robot_arm_sensor.ipynb`にて解説しています。）  

![](../images/RobotArm/2d_robot_arm.png)

In [3]:
# ロボットの読み込み
armStartPos = [0, 0, 0.1]  # 初期位置(x,y,z)を設定
armStartOrientation = pybullet.getQuaternionFromEuler([0,0,0])  # 初期姿勢(roll, pitch, yaw)を設定
armId = pybullet.loadURDF("../urdf/simple2d_arm.urdf",armStartPos, armStartOrientation, useFixedBase=True) # ロボットが倒れないように、useFixedBase=Trueでルートのリンクを固定

# GUIモードの際のカメラの位置などを設定
cameraDistance = 1.5
cameraYaw = 180.0 # deg
cameraPitch = -10 # deg
cameraTargetPosition = [0.0, 0.0, 1.0]
pybullet.resetDebugVisualizerCamera(cameraDistance, cameraYaw, cameraPitch, cameraTargetPosition)

b3Printf: b3Warning[examples/Importers/ImportURDFDemo/BulletUrdfImporter.cpp,126]:

b3Printf: No inertial data for link, using mass=1, localinertiadiagonal = 1,1,1, identity local inertial frame
b3Printf: b3Warning[examples/Importers/ImportURDFDemo/BulletUrdfImporter.cpp,126]:

b3Printf: target_position_vertual_link


### <a id='toc2_2_4_'></a>[三角関数による順運動学の関数の定義](#toc0_)

「理論」の章で解説した「三角関数による順運動学」の関数を定義します．

今回の場合、式(1)(2)及び、式(3)(4)は以下の関数で実装できます。


In [4]:
import numpy as np
def forward_kinematics2d(linkLength, theta):
    """
    2次元平面における（三角関数による）順運動学を求める
    Parameters
    ----------
    linkLength : float
        リンクの長さ
    theta : float
        回転角度(rad)

    Returns
    -------
    x: リンク先端座標(x)
    y: リンク先端座標(y)
    """
    x = linkLength*np.cos(theta)
    y = linkLength*np.sin(theta)
    return x, y

<br>

### <a id='toc2_2_5_'></a>[三角関数による順運動学の実行](#toc0_)

次に、以下コードを実行すると、「`link1_angle_deg`と`link2_angle_deg`に設定した角度」で順運動学が計算され、計算結果の「エンドエフェクタの位置」が画面上に表示されます。

また、各関節の角度が`link1_angle_deg`と`link2_angle_deg`に設定されるので、順運動の結果が実際のエンドエフェクタの位置と一致していることを確認してみてください。




In [6]:
import math
link1Length = 0.5   # link1の長さ(「simple2d_arm.urdf」のlink1の長さ)
link2Length = 0.55  # link2の長さ(「simple2d_arm.urdf」のlink2+force_sensor_linkの長さ)

# ここの値を変えると、結果が変わります##########
link1AngleDeg = 30
link2AngleDeg = 90
###############################################

link1AngleRad = math.radians(link1AngleDeg)
link2AngleRad = math.radians(link2AngleDeg)

# 三角関数による順運動学を用いて（初期）手先位置P_currentを計算
# link1の根本から見た時の，link1の先端位置
x1To2, y1To2 = forward_kinematics2d(link1Length, link1AngleRad)

# link2の根本から見た時の，link2の先端位置（=エンドエフェクタの位置）
x2ToE, y2ToE = forward_kinematics2d(link2Length, link1AngleRad+link2AngleRad)

# link1の根本（原点座標）から見た時の，エンドエフェクタの位置
xe = x1To2 + x2ToE
ye = y1To2 + y2ToE

# ターミナルに結果を表示
print(f"x, y = ({xe}, {ye})")

# 画面上に結果を表示
textPosition = [0.5, 0.0, 2.0]
lifeTime = 10.0 # 表示期間（秒）
pybullet.addUserDebugText(f"x, y = ({xe:.2f}, {ye:.2f})", textPosition, textSize=2, lifeTime=lifeTime)

# 実際に関節を動かして、順運動学の結果と等しいかを確認
pybullet.resetJointState(armId, 0, link1AngleRad)
pybullet.resetJointState(armId, 1, link2AngleRad)
pybullet.stepSimulation()

x, y = (0.15801270189221944, 0.7263139720814413)


()